In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

data_dir = Path(r"C:\Python_Sripts\VNA\Data")
cst_file = data_dir / "Z002_Impedances_CST3.txt"
dm_vna_file = data_dir / "dm_VNA.csv"
cm_vna_file = data_dir / "cm_VNA.csv"

In [7]:
%matplotlib qt

In [8]:
# Read CST data
with open(cst_file, 'r') as f:
    lines = f.readlines()

cst_data = {'DM': {'freq': [], 'Z': []}, 'CM': {'freq': [], 'Z': []}}
current_mode = None

for line in lines:
    # Check header lines for mode indicators
    if line.startswith('#'):
        if 'Z8,8' in line:
            current_mode = 'DM'
        elif 'Z5,5' in line:
            current_mode = 'CM'
        continue
    
    # Skip empty lines
    line = line.strip()
    if not line:
        continue
    
    # Parse data lines
    if current_mode:
        try:
            parts = line.split()  # or use .split('\t') for tab-separated
            if len(parts) >= 2:
                cst_data[current_mode]['freq'].append(float(parts[0]))
                cst_data[current_mode]['Z'].append(float(parts[1]))
        except ValueError:
            continue

for mode in ['DM', 'CM']:
    cst_data[mode]['freq'] = np.array(cst_data[mode]['freq'])
    cst_data[mode]['Z'] = np.array(cst_data[mode]['Z'])

In [9]:
# Read VNA data - skip header lines starting with !
skip_rows_dm = sum(1 for line in open(dm_vna_file) if line.startswith('!'))
skip_rows_cm = sum(1 for line in open(cm_vna_file) if line.startswith('!'))

dm_data = pd.read_csv(dm_vna_file, skiprows=skip_rows_dm)
cm_data = pd.read_csv(cm_vna_file, skiprows=skip_rows_cm)

# Extract frequency and impedance (data is in dB, convert to linear)
dm_freq = dm_data.iloc[:, 0].values
dm_z_db = dm_data.iloc[:, 1].values
dm_z = 10 ** (dm_z_db / 20)  # Convert dB to linear Ohms

cm_freq = cm_data.iloc[:, 0].values
cm_z_db = cm_data.iloc[:, 1].values
cm_z = 10 ** (cm_z_db / 20)  # Convert dB to linear Ohms

In [10]:
# Plot comparison
from matplotlib.ticker import EngFormatter

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# DM (Differential Mode)
ax1.loglog(cst_data['DM']['freq'], cst_data['DM']['Z'], 'b-', linewidth=2, label='CST Simulation')
ax1.loglog(dm_freq, dm_z, 'r--', linewidth=1.5, label='VNA Measurement')
ax1.xaxis.set_major_formatter(EngFormatter(unit='Hz'))
ax1.set_xlabel('Frequency')
ax1.set_ylabel('Impedance (Ω)')
ax1.set_title('Differential Mode (DM) Impedance')
ax1.grid(True, which='both', alpha=0.3)
ax1.legend()

# CM (Common Mode)
ax2.loglog(cst_data['CM']['freq'], cst_data['CM']['Z'], 'b-', linewidth=2, label='CST Simulation')
ax2.loglog(cm_freq, cm_z, 'r--', linewidth=1.5, label='VNA Measurement')
ax2.xaxis.set_major_formatter(EngFormatter(unit='Hz'))
ax2.set_xlabel('Frequency')
ax2.set_ylabel('Impedance (Ω)')
ax2.set_title('Common Mode (CM) Impedance')
ax2.grid(True, which='both', alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()